In [1]:
import tensorflow as tf

import numpy as np
import os
import time
import re


In [2]:
#I editted the text so that only stuff written by Cervantes remains.
text = open('pg996.txt').read()
print(text[:250])

Idle reader: thou mayest believe me without any oath that I would this
book, as it is the child of my brain, were the fairest, gayest, and
cleverest that could be imagined. But I could not counteract Nature’s
law that everything shall beget its like


In [3]:
#removes text representing placeholders for images.
while text.find('.jpg')!=-1:
    pos= text.find('.jpg')
    text = text.replace(text[pos-6:pos+25], '')

#need to replace /n with " " but not /n/n (or longer)
pattern = r'(?<!\n)\n(?!\n)'
text = re.sub(pattern, ' ', text)

In [4]:
# The unique characters in the file
vocab = sorted(set(text))
vocab_size=len(vocab)
print(f'{vocab_size} unique characters')

91 unique characters


In [5]:
#creates a mapping that assigns all characters in the text an index.
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None)
chars_from_ids = tf.keras.layers.StringLookup(vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

#using perhaps a better methods
vocab_size = len(ids_from_chars.get_vocabulary())

In [6]:
#transforms the text into an array of unicode characters and then into ids.
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))

ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [7]:
seq_length = 100
# The batch method lets you easily convert these individual characters to sequences of the desired size.
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'\xef\xbb\xbf' b'I' b'd' b'l' b'e' b' ' b'r' b'e' b'a' b'd' b'e' b'r'
 b':' b' ' b't' b'h' b'o' b'u' b' ' b'm' b'a' b'y' b'e' b's' b't' b' '
 b'b' b'e' b'l' b'i' b'e' b'v' b'e' b' ' b'm' b'e' b' ' b'w' b'i' b't'
 b'h' b'o' b'u' b't' b' ' b'a' b'n' b'y' b' ' b'o' b'a' b't' b'h' b' '
 b't' b'h' b'a' b't' b' ' b'I' b' ' b'w' b'o' b'u' b'l' b'd' b' ' b't'
 b'h' b'i' b's' b' ' b'b' b'o' b'o' b'k' b',' b' ' b'a' b's' b' ' b'i'
 b't' b' ' b'i' b's' b' ' b't' b'h' b'e' b' ' b'c' b'h' b'i' b'l' b'd'
 b' ' b'o' b'f' b' ' b'm'], shape=(101,), dtype=string)


2024-06-03 11:45:20.899288: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [8]:
#For training you'll need a dataset of (input, label) pairs. Where input and label are sequences. At each time step the input is the current character and the label is the next character.

#Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [9]:
dataset = sequences.map(split_input_target)

for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'\xef\xbb\xbfIdle reader: thou mayest believe me without any oath that I would this book, as it is the child of '
Target: b'Idle reader: thou mayest believe me without any oath that I would this book, as it is the child of m'


2024-06-03 11:45:20.975632: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [10]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

## Building the Model

In [11]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Embedding, GRU, Dropout

In [20]:
def Basic_RNN(hidden_units, vocab_size,embedding_dim):
    model = Sequential()
    #model.add(Input(shape=input_shape))
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
    model.add(SimpleRNN(hidden_units, activation='tanh',return_sequences=True))
    model.add(Dense(vocab_size))

    loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(loss=loss, optimizer='adam')
    return model

def Basic_RNN_v2(hidden_units, vocab_size,embedding_dim,dropout_rate=0.0,recurrent_dropout_rate=0.0):
    model = Sequential()
    #model.add(Input(shape=input_shape))
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
    model.add(SimpleRNN(hidden_units, activation='tanh',return_sequences=True,
                        dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
    model.add(SimpleRNN(hidden_units, activation='tanh',return_sequences=True,
                        dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
    model.add(SimpleRNN(hidden_units, activation='tanh',return_sequences=True,
                        dropout=dropout_rate,recurrent_dropout=recurrent_dropout_rate))
    model.add(Dense(vocab_size))

    loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(loss=loss, optimizer='adam')
    return model

def GRU_RNN(hidden_units, vocab_size,embedding_dim):
    model = Sequential()
    #model.add(Input(shape=input_shape))
    model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
    model.add(GRU(hidden_units, activation='tanh',return_sequences=True))
    model.add(Dense(vocab_size))

    loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(loss=loss, optimizer='adam')
    return model

In [23]:
embedding_dim=256
hidden_units=256

basic_model = Basic_RNN(hidden_units, vocab_size,embedding_dim)
EPOCHS=40
basic_model.fit(dataset,epochs=EPOCHS)

Epoch 1/40


337/337 ━━━━━━━━━━━━━━━━━━━━ 45s 128ms/step - loss: 2.4605
Epoch 2/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 43s 126ms/step - loss: 1.7115
Epoch 3/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 43s 124ms/step - loss: 1.5591
Epoch 4/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 42s 123ms/step - loss: 1.4832
Epoch 5/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 44s 127ms/step - loss: 1.4334
Epoch 6/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 43s 125ms/step - loss: 1.4047
Epoch 7/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 42s 120ms/step - loss: 1.3784
Epoch 8/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 1624s 5s/step - loss: 1.3576
Epoch 9/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 669s 2s/step - loss: 1.3420
Epoch 10/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 44s 126ms/step - loss: 1.3288
Epoch 11/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 43s 125ms/step - loss: 1.3189
Epoch 12/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 43s 124ms/step - loss: 1.3089
Epoch 13/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 43s 124ms/step - loss: 1.3018
Epoch 14/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 43s 124ms/step - loss: 1.2929
Epoch 15/40
337/337 ━━━━━━━━━

In [24]:
hidden_units=128
v2_model_nodropout = Basic_RNN_v2(hidden_units, vocab_size,embedding_dim)
EPOCHS=40
v2_model_nodropout.fit(dataset,epochs=EPOCHS)

Epoch 1/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 50s 141ms/step - loss: 2.7103
Epoch 2/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 57s 167ms/step - loss: 1.7589
Epoch 3/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 58s 170ms/step - loss: 1.5854
Epoch 4/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 58s 170ms/step - loss: 1.4971
Epoch 5/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 51s 148ms/step - loss: 1.4452
Epoch 6/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 51s 149ms/step - loss: 1.4083
Epoch 7/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 51s 150ms/step - loss: 1.3810
Epoch 8/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 49s 143ms/step - loss: 1.3602
Epoch 9/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 50s 145ms/step - loss: 1.3409
Epoch 10/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 51s 149ms/step - loss: 1.3293
Epoch 11/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 51s 148ms/step - loss: 1.3176
Epoch 12/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 51s 148ms/step - loss: 1.3107
Epoch 13/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 54s 158ms/step - loss: 1.3008
Epoch 14/40
337/337 ━━━━━━━━━━━━━━━━━━━━ 54s 156ms/step - loss: 1.2922
Epoch 15/40
337

## Test Model

In [25]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

  @tf.function
  def generate_one_step(self, inputs):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()
    
    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits = self.model(input_ids)[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
   
    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars

In [30]:
one_step_model = OneStep(basic_model, chars_from_ids, ids_from_chars, 0.5)

next_char = tf.constant(['Don Quixote '])
result = [next_char]

for n in range(100):
  next_char = one_step_model.generate_one_step(next_char)
  result.append(next_char)

result = tf.strings.join(result)
print(result)

tf.Tensor([b'Don Quixote o an are the hed wive s f wan I f n ore the athe the s tof illld se hed has the and thatithand ort b'], shape=(1,), dtype=string)


In [46]:
V2one_step_model = OneStep(v2_model_nodropout, chars_from_ids, ids_from_chars, 0.4)

next_char = tf.constant(['He had not gone far, when out of a thicket on his right there seemed to come feeble '])
result = [next_char]

for n in range(100):
  next_char = V2one_step_model.generate_one_step(next_char)
  result.append(next_char)

result = tf.strings.join(result)
print(result)

tf.Tensor([b'He had not gone far, when out of a thicket on his right there seemed to come feeble s anchous and t tore an che t the theand wo the he thant an t s theanced anche the he the t and s s '], shape=(1,), dtype=string)


In [453]:
GRU_model.save('GRU_model_01062024.keras')